In [7]:
##https://www.linkedin.com/pulse/convert-netcdf4-file-geotiff-using-python-chonghua-yin
from osgeo import gdal, osr, gdal_array
# import xarray as xr
import numpy as np

def GetnetCDFInfobyName(in_filename, var_name):
    '''
    Function to read netCDF data info
    '''
    
    #Open netCDF file
    src_ds = gdal.Open(in_filename)
    if src_ds is None:
        print('Open failed')
        sys.exit()
    else:
        
        #if more than one sub-dataset exists in the netCDF
        subdataset = 'NETCDF:"'+in_filename+'":/science/grids/data/'+var_name
        src_ds_sd = gdal.Open(subdataset)
        
        #begin reading the subdataset
        NDV = src_ds_sd.GetRasterBand(1).GetNoDataValue()
        xsize = src_ds_sd.RasterXSize
        ysize = src_ds_sd.RasterYSize
        GeoT = src_ds_sd.GetGeoTransform()
        Projection = osr.SpatialReference()
        Projection.ImportFromWkt(src_ds_sd.GetProjectionRef())
        data = src_ds_sd.GetRasterBand(1).ReadAsArray()
        #close the dataset and file
        src_ds_sd = None
        src_ds = None
        
        #read data using xarray
#         xr_ensemble = xr.open_dataset(in_filename)
#         data = xr_ensemble[var_name]
#         data = np.ma.masked_array(data, mask=data==NDV, fill_value=NDV)
        
        
        return NDV,xsize,ysize,GeoT,Projection,data
    
def saveGDALfile(new_filename,Array,NDV,xsize,ysize,GeoT,Projection,file_format):
    '''
    Create new file using GDAL
    '''
    
    DataType = gdal_array.NumericTypeCodeToGDALTypeCode(Array.dtype)
    
    if type(DataType)!=np.int:
        if DataType.startswith('gdal.GDT_') == False:
            DataType = eval('gdal.GDT_'+DataType)
            
    NewFileName = new_filename
#    zsize = Array.shape[0]
    
    #Create a driver
    driver = gdal.GetDriverByName(file_format)
    
    #Set NaN to original No Data Value
    Array[np.isnan(Array)] = NDV
    
    #Create the dataset with zsize bands
    Dataset = driver.Create(NewFileName,xsize,ysize,1,DataType)
    Dataset.SetGeoTransform(GeoT)
    Dataset.SetProjection(Projection.ExportToWkt())
    Dataset.GetRasterBand(1).WriteArray(Array)

    #Write each slice of the array along the zsize
#     for i in range(0,zsize):
#         Dataset.GetRasterBand(i+1).WriteArray(Array[i])
#         Dataset.GetRasterBand(i+1).SetNoDataValue(NDV)
        
    Dataset.FlushCache()
    
    return NewFileName
        
        

In [12]:
infile = 'S1-GUNW-A-R-077-tops-20190222_20190210-231605-42666N_40796N-PP-d75b-v2_0_1.nc'
varname = 'unwrappedPhase'

NDV,xsize,ysize,GeoT,Projection,data = GetnetCDFInfobyName(infile, varname)
outfile = saveGDALfile('unwrappedPhase.dat', data, NDV,xsize,ysize,GeoT,Projection,'ENVI')

In [10]:
gdal.Info('unwrappedPhase.dat')

'Driver: ENVI/ENVI .hdr Labelled\nFiles: unwrappedPhase.dat\n       unwrappedPhase.hdr\nSize is 4232, 2246\nCoordinate System is:\nGEOGCRS["GCS_unknown",\n    DATUM["D_unnamed",\n        ELLIPSOID["Spheroid",6378137,298.257223563,\n            LENGTHUNIT["metre",1,\n                ID["EPSG",9001]]]],\n    PRIMEM["Greenwich",0,\n        ANGLEUNIT["Degree",0.0174532925199433]],\n    CS[ellipsoidal,2],\n        AXIS["longitude",east,\n            ORDER[1],\n            ANGLEUNIT["Degree",0.0174532925199433]],\n        AXIS["latitude",north,\n            ORDER[2],\n            ANGLEUNIT["Degree",0.0174532925199433]]]\nData axis to CRS axis mapping: 1,2\nOrigin = (-81.866249999999994,42.667083333333402)\nPixel Size = (0.000833333333333,-0.000833333333333)\nMetadata:\n  Band_1=Band 1\nImage Structure Metadata:\n  INTERLEAVE=BAND\nCorner Coordinates:\nUpper Left  ( -81.8662500,  42.6670833) ( 81d51\'58.50"W, 42d40\' 1.50"N)\nLower Left  ( -81.8662500,  40.7954167) ( 81d51\'58.50"W, 40d47\'43